# Get Information About Current Clan War
## Init

In [ ]:
#r "nuget: ClashOfClans"

In [ ]:
using System;
using System.IO;
using ClashOfClans;
using ClashOfClans.Core;
using ClashOfClans.Models;
using Newtonsoft.Json.Linq;

var settings = JObject.Parse(File.ReadAllText("settings.json"));
var token = (string)settings["Token"];
var clanTag = (string)settings["ClanTag"];

var coc = new ClashOfClansClient(token);

## Get Clan Information

In [ ]:
try {
    var clan = await coc.Clans.GetClanAsync(clanTag);

    if (clan.IsWarLogPublic == false) {
        throw new System.NotSupportedException("War log must be public to access current war information!");
    }
}
catch (ClashOfClansException ex) {
    Console.WriteLine(ex.Error.Message);
}

## Get War Information

In [ ]:
var currentWar = await coc.Clans.GetCurrentWarAsync(clanTag);

Console.WriteLine($"War state: {currentWar.State}");
Console.WriteLine($"Preparation start time @ {currentWar.PreparationStartTime.ToLocalTime()}");
Console.WriteLine($"War start time @ {currentWar.StartTime.ToLocalTime()}");
Console.WriteLine($"War end time @ {currentWar.EndTime.ToLocalTime()}");

## Register a Formatter for `ClanWar`

In [ ]:
using Microsoft.DotNet.Interactive.Formatting;

string GetAttack(ClanWarAttackList attacks, int attackNumber, ClanWarMemberList members) {
    if (attacks == null) {
        return string.Empty;
    }

    if (attacks.Count < attackNumber) {
        return string.Empty;
    }

    var attack = attacks[attackNumber-1];
    var opponent = members.Single(m => m.Tag == attack.DefenderTag);
    return $"{new String('\u2605', attack.Stars)}, {attack.DestructionPercentage:D3}%, #{opponent.MapPosition:D2}";
}

Formatter.ResetToDefault();
Formatter.Register<ClanWar>((clanWar, writer) => {
    writer.Write("<table>");
    writer.Write("<th>Position</th>");
    writer.Write("<th>Player Name</th>");
    writer.Write("<th>1st Attack</th>");
    writer.Write("<th>2nd Attack</th>");
    writer.Write("<th>Opponent Name</th>");
    writer.Write("<th>1st Attack</th>");
    writer.Write("<th>2nd Attack</th>");
    writer.Write("</tr>");
    foreach (var member in clanWar.Clan.Members.OrderBy(m => m.MapPosition)) {
        var opponent = clanWar.Opponent.Members.Single(m => m.MapPosition == member.MapPosition);
        writer.Write("<tr>");
        writer.Write($"<td>{member.MapPosition}.</td>");
        writer.Write($"<td>{member.Name}</td>");
        writer.Write($"<td>{GetAttack(member.Attacks, 1, clanWar.Opponent.Members)}</td>");
        writer.Write($"<td>{GetAttack(member.Attacks, 2, clanWar.Opponent.Members)}</td>");
        writer.Write($"<td>{opponent.Name}</td>");
        writer.Write($"<td>{GetAttack(opponent.Attacks, 1, clanWar.Clan.Members)}</td>");
        writer.Write($"<td>{GetAttack(opponent.Attacks, 2, clanWar.Clan.Members)}</td>");
        writer.Write("</tr>");
    }
    writer.Write("<tr>");
    writer.Write($"<td></td>");
    writer.Write($"<td></td>");
    writer.Write($"<td>{clanWar.Clan.Stars}\u2605, {clanWar.Clan.DestructionPercentage}%</td>");
    writer.Write($"<td></td>");
    writer.Write($"<td></td>");
    writer.Write($"<td>{clanWar.Opponent.Stars}\u2605, {clanWar.Opponent.DestructionPercentage}%</td>");
    writer.Write($"<td></td>");
    writer.Write("</tr>");
    writer.Write("</table>");
}, mimeType: "text/html");

## War Details

In [ ]:
currentWar